In [1]:
import tkinter as tk
from tkinter import messagebox

class Book:
    def __init__(self, id, name, quantity):
        self.id = id
        self.name = name
        self.quantity = int(quantity)

class User:
    def __init__(self, id, name):
        self.id = id
        self.name = name

class Admin:
    def __init__(self):
        self.books = []
        self.users = {}

    def add_book(self, book_id, book_name, book_quantity):
        for book in self.books:
            if book.id == book_id:
                return "This book already exists"
        new_book = Book(book_id, book_name, book_quantity)
        self.books.append(new_book)
        return "Book added successfully"

    def print_all_books(self):
        lst_all_books = [book.name for book in self.books]
        return ", ".join(lst_all_books) if lst_all_books else "There are no books"

    def search_for_book(self, search):
        found_book = [book.name for book in self.books if book.name.upper() == search.upper()]
        return found_book

    def add_user(self, user_id, user_name):
        if user_id in self.users:
            return "This user already exists"
        new_user = User(user_id, user_name)
        self.users[new_user] = None
        return "User added successfully"

    def borrow_book(self, user_name, book_name):
        found_book = ''.join(self.search_for_book(book_name))
        if not found_book:
            return "This book does not exist"
        available_books = [book for book in self.books if book.name == found_book and book.quantity > 0]
        if not available_books:
            return "Insufficient quantity!"
        available_books[0].quantity -= 1
        for user, values in self.users.items():
            if user.name.lower() == user_name.lower():
                if values is None:
                    self.users[user] = [book_name]
                else:
                    self.users[user].append(book_name)
                return f"The user {user_name} borrowed {found_book}"
        return "There is no user with this name"

    def return_book(self, user_name, book_name):
        found_book = ''.join(self.search_for_book(book_name))
        if not found_book:
            return "This book does not exist"
        available_books = [book for book in self.books if book.name == found_book]
        if not available_books:
            return "There is no book with this name"
        available_books[0].quantity += 1
        for user, values in self.users.items():
            if user.name.lower() == user_name.lower():
                if values and book_name in values:
                    if len(values) == 1:
                        self.users[user] = None
                    else:
                        self.users[user].remove(book_name)
                    return f"The user {user_name} returned {found_book}"
        return "There is no user with this name"

    def print_users_borrowed(self):
        users_found = [user for user in self.users if self.users[user] is not None]
        return "\n".join([f"The user {user.name} borrowed the books {' and '.join(self.users[user])}" for user in users_found]) if users_found else "There are no users who borrowed any books"

    def print_all_users(self):
        all_users = [user.name for user in self.users]
        return ", ".join(all_users) if all_users else "There are no users"

class LibraryApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Library Management System")
        self.admin = Admin()

        # Main Menu
        self.menu_frame = tk.Frame(root)
        self.menu_frame.pack(pady=20)

        tk.Button(self.menu_frame, text="Add Book", command=self.add_book_gui).grid(row=0, column=0, padx=10, pady=5)
        tk.Button(self.menu_frame, text="Print Library Books", command=self.print_all_books).grid(row=0, column=1, padx=10, pady=5)
        tk.Button(self.menu_frame, text="Search Book", command=self.search_book_gui).grid(row=1, column=0, padx=10, pady=5)
        tk.Button(self.menu_frame, text="Add User", command=self.add_user_gui).grid(row=1, column=1, padx=10, pady=5)
        tk.Button(self.menu_frame, text="Borrow Book", command=self.borrow_book_gui).grid(row=2, column=0, padx=10, pady=5)
        tk.Button(self.menu_frame, text="Return Book", command=self.return_book_gui).grid(row=2, column=1, padx=10, pady=5)
        tk.Button(self.menu_frame, text="Print Users Borrowed", command=self.print_users_borrowed).grid(row=3, column=0, padx=10, pady=5)
        tk.Button(self.menu_frame, text="Print All Users", command=self.print_all_users).grid(row=3, column=1, padx=10, pady=5)

    def add_book_gui(self):
        def submit():
            book_id = id_entry.get()
            book_name = name_entry.get()
            book_quantity = quantity_entry.get()
            message = self.admin.add_book(book_id, book_name, book_quantity)
            messagebox.showinfo("Info", message)
            add_book_window.destroy()

        add_book_window = tk.Toplevel(self.root)
        add_book_window.title("Add Book")

        tk.Label(add_book_window, text="Book ID:").grid(row=0, column=0)
        id_entry = tk.Entry(add_book_window)
        id_entry.grid(row=0, column=1)

        tk.Label(add_book_window, text="Book Name:").grid(row=1, column=0)
        name_entry = tk.Entry(add_book_window)
        name_entry.grid(row=1, column=1)

        tk.Label(add_book_window, text="Book Quantity:").grid(row=2, column=0)
        quantity_entry = tk.Entry(add_book_window)
        quantity_entry.grid(row=2, column=1)

        tk.Button(add_book_window, text="Add Book", command=submit).grid(row=3, column=1, pady=10)

    def print_all_books(self):
        messagebox.showinfo("All Books", self.admin.print_all_books())

    def search_book_gui(self):
        def search():
            query = query_entry.get()
            result = ', '.join(self.admin.search_for_book(query))
            messagebox.showinfo("Search Results", result if result else "This book does not exist")
            search_window.destroy()

        search_window = tk.Toplevel(self.root)
        search_window.title("Search Book")

        tk.Label(search_window, text="Enter book name:").grid(row=0, column=0)
        query_entry = tk.Entry(search_window)
        query_entry.grid(row=0, column=1)

        tk.Button(search_window, text="Search", command=search).grid(row=1, column=1, pady=10)

    def add_user_gui(self):
        def submit():
            user_id = id_entry.get()
            user_name = name_entry.get()
            message = self.admin.add_user(user_id, user_name)
            messagebox.showinfo("Info", message)
            add_user_window.destroy()

        add_user_window = tk.Toplevel(self.root)
        add_user_window.title("Add User")

        tk.Label(add_user_window, text="User ID:").grid(row=0, column=0)
        id_entry = tk.Entry(add_user_window)
        id_entry.grid(row=0, column=1)

        tk.Label(add_user_window, text="User Name:").grid(row=1, column=0)
        name_entry = tk.Entry(add_user_window)
        name_entry.grid(row=1, column=1)

        tk.Button(add_user_window, text="Add User", command=submit).grid(row=2, column=1, pady=10)

    def borrow_book_gui(self):
        def submit():
            user_name = user_entry.get()
            book_name = book_entry.get()
            message = self.admin.borrow_book(user_name, book_name)
            messagebox.showinfo("Info", message)
            borrow_window.destroy()

        borrow_window = tk.Toplevel(self.root)
        borrow_window.title("Borrow Book")

        tk.Label(borrow_window, text="User Name:").grid(row=0, column=0)
        user_entry = tk.Entry(borrow_window)
        user_entry.grid(row=0, column=1)

        tk.Label(borrow_window, text="Book Name:").grid(row=1, column=0)
        book_entry = tk.Entry(borrow_window)
        book_entry.grid(row=1, column=1)

        tk.Button(borrow_window, text="Borrow", command=submit).grid(row=2, column=1, pady=10)

    def return_book_gui(self):
        def submit():
            user_name = user_entry.get()
            book_name = book_entry.get()
            message = self.admin.return_book(user_name, book_name)
            messagebox.showinfo("Info", message)
            return_window.destroy()

        return_window = tk.Toplevel(self.root)
        return_window.title("Return Book")

        tk.Label(return_window, text="User Name:").grid(row=0, column=0)
        user_entry = tk.Entry(return_window)
        user_entry.grid(row=0, column=1)

        tk.Label(return_window, text="Book Name:").grid(row=1, column=0)
        book_entry = tk.Entry(return_window)
        book_entry.grid(row=1, column=1)

        tk.Button(return_window, text="Return", command=submit).grid(row=2, column=1, pady=10)

    def print_users_borrowed(self):
        messagebox.showinfo("Users Borrowed Books", self.admin.print_users_borrowed())

    def print_all_users(self):
        messagebox.showinfo("All Users", self.admin.print_all_users())

# Running the application
root = tk.Tk()
app = LibraryApp(root)
root.mainloop()